<a href="https://colab.research.google.com/github/aldolipani/SDAE19/blob/master/SDAE_MLP_CNN_Tutorial_on_MINST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install PyTorch

In [0]:
!pip install torch
!pip install torchvision

## Import Dependencies

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm_notebook
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np

## Download MNIST Dataset

In [0]:
train_set = datasets.MNIST(root = './data', train = True, transform = transforms.ToTensor(), download = True)
test_set  = datasets.MNIST(root = './data', train = False, transform = transforms.ToTensor())

print()
print("trainset:\n", train_set)
print("testset:\n", test_set)

batch_size = 4 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader =  torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

## Analyse MNIST Dataset

Let's start by visualising the dataset:

In [0]:
# functions to show an image
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
print("Input values (X):")
imshow(torchvision.utils.make_grid(images))
# print labels
print("Labels:")
print(' '.join('%9s' % str(labels[j].item()) for j in range(4)))

Let's now print some statistics about the dataset

In [0]:
print("Shape of the input: ", images.shape)
print("Shape of the output:", labels.shape)

# Hyperparameters based on the Dataset

In [0]:
input_size = 784  # img_size = (28,28) ---> 28*28=784 in total
num_classes = 10  # number of output classes discrete range [0,9]

# MLP in Pytorch

Let's first define the model:

In [0]:
class MLP(nn.Module):
  def __init__(self, input_size, hidden_sizes, output_size):
    super(MLP, self).__init__()
    self.layers = nn.ModuleList()
    next_hidden_size = input_size
    for hidden_size in hidden_sizes:
      self.layers.append(nn.Linear(next_hidden_size, hidden_size))
      next_hidden_size = hidden_size
    self.output = nn.Linear(next_hidden_size, output_size)

  def forward(self, x):
    out = x
    for layer in self.layers:
        out = layer(out)
        out = F.relu(out)
    out = self.output(out)
    return out

Then, we instantiate the model:

In [0]:
model = MLP(input_size, [400, 400, 10], num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Optimizer

In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training

In [0]:
num_epochs = 3 # how many times you want to train on the dataset

criterium = nn.CrossEntropyLoss() # loss function

batch_size = 50
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader =  torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

model.train() # set the model to training mode

running_loss = 0
training_loss = []
#testing_loss = []
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    xs = images.view(-1, input_size).to(device)
    ys = labels.to(device)
    
    optimizer.zero_grad() # reset the gradients
    pred_ys = model(xs) # generate the predictions
    loss = criterium(pred_ys, ys) # compute the loss
    loss.backward() # backpropagation
    optimizer.step() # optmizes here
    
    running_loss += loss.item()
    if (i+1) % 100 == 0:
      running_loss /= 100
      training_loss.append(running_loss)
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'%(epoch+1, num_epochs, i+1, len(train_set)//batch_size, running_loss))
      running_loss = 0
      #testing_loss.append(get_test_loss(model))

# Analyse Losses

In [0]:
plt.plot(training_loss)
#if testing_loss:
#  plt.plot(testing_loss)
plt.show()

# Analyse the loss on the Test Set

In [0]:
def get_test_loss(model):
  model.eval()
  
  running_loss = 0
  for i, (images, labels) in enumerate(test_loader):
    xs = images.view(-1, input_size).to(device)
    ys = labels
    
    pred_ys = model(xs)
    loss = criterium(pred_ys, ys)
    running_loss += loss.item()

  model.train()
  return running_loss/len(test_data)

#Evaluating the Classfier

In [0]:
model.eval()

def accuracy(data_loader):
  correct = 0.0
  total = 0.0
  with torch.no_grad():
    for images, labels in tqdm_notebook(data_loader):
      xs = images.view(-1, input_size).to(device)
      ys = labels

      pred_ys = model(xs).detach().cpu()
      _, pred_ys = torch.max(pred_ys, 1)
      correct += (pred_ys == ys).sum().numpy()
      total += labels.size(0)
      
  return correct/total

train_accuracy = accuracy(train_loader)
print('Train accuracy: {:.3f}'.format(train_accuracy))

test_accuracy = accuracy(test_loader)
print('Test accuracy: {:.3f}'.format(test_accuracy))

#CNN

In [0]:
class CNN(nn.Module):
  def __init__(self, conv_hidden_sizes:list, linear_hidden_sizes, output_size):
    super(CNN, self).__init__()
    self.conv_layers = nn.ModuleList()
    next_conv_size = 1
    for conv_hidden_size in conv_hidden_sizes:
      conv = nn.Conv2d(next_conv_size, conv_hidden_size, kernel_size=5)
      self.conv_layers.append(conv)
      next_conv_size = conv_hidden_size
    self.linear_layers = nn.ModuleList()
    next_linear_size = 3*3*next_conv_size
    for linear_hidden_size in linear_hidden_sizes:
      linear = nn.Linear(next_linear_size, linear_hidden_size)
      self.linear_layers.append(linear)
      next_linear_size = linear_hidden_size
    self.output = nn.Linear(linear_hidden_size, output_size)

  def forward(self, x):
    out = x
    for i, layer in enumerate(self.conv_layers):
      out = layer(out)
      if i > 0:
        out = F.max_pool2d(out, 2)
      out = F.relu(out)    
    out = out.view(-1,3*3*64)
    for layer in self.linear_layers:
      out = layer(out)
      out = F.relu(out)
    out = self.output(out)
    return out

model = CNN([32, 32, 64], [256], 10)

model = model.to(device)

model

In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [0]:
num_epochs = 3 # how many times you want to train on the dataset

criterium = nn.CrossEntropyLoss() # loss function

batch_size = 50
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader =  torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

model.train() # set the model to training mode

running_loss = 0
training_loss = []
#testing_loss = []
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    xs = images.to(device)
    ys = labels.to(device)
    
    optimizer.zero_grad() # reset the gradients
    pred_ys = model(xs) # generate the predictions
    loss = criterium(pred_ys, ys) # compute the loss
    loss.backward() # backpropagation
    optimizer.step() # optmizes here
    
    running_loss += loss.item()
    if (i+1) % 100 == 0:
      running_loss /= 100
      training_loss.append(running_loss)
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'%(epoch+1, num_epochs, i+1, len(train_set)//batch_size, running_loss))
      running_loss = 0
      #testing_loss.append(get_test_loss(model))

In [0]:
model.eval()

def accuracy(data_loader):
  correct = 0.0
  total = 0.0
  with torch.no_grad():
    for images, labels in tqdm_notebook(data_loader):
      xs = images.to(device)
      ys = labels

      pred_ys = model(xs).detach().cpu()
      _, pred_ys = torch.max(pred_ys, 1)
      correct += (pred_ys == ys).sum().numpy()
      total += labels.size(0)
      
  return correct/total


train_accuracy = accuracy(train_loader)
print('Train accuracy: {:.3f}'.format(train_accuracy))

test_accuracy = accuracy(test_loader)
print('Test accuracy: {:.3f}'.format(test_accuracy))